# STARDIS

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import ndimage

from tardis.simulation import Simulation
from tardis.io.config_reader import Configuration
from tardis.io.atom_data import AtomData

from astropy import units as u, constants as const

from stardis.plasma import AlphaLine, HMinusDensity, TracingNus, create_splasma
from stardis.opacities import calc_tau_h_minus, calc_tau_e, calc_tau_nus, calc_hminus_density, calc_tau_photo
from stardis.io import read_marcs_to_fv
from stardis.raytrace import bb_nu, bb_lambda, calc_weights

## Sun Spectrum

In [ ]:
sun_spec = pd.read_csv('data/solar_data/NewGuey2003.txt', skiprows=20, delim_whitespace=True, names=['wave', 'flux'])
sun_spec['wave'] *= 10
sun_spec['flux'] = u.Quantity(sun_spec['flux'].to_numpy(), "W/(m2 nm)").to("erg/(s cm2 AA)").value
plt.plot(sun_spec.wave, sun_spec.flux)
plt.xlim(1000, 9000)

## Model & Atomic Data

In [ ]:
adata = AtomData.from_hdf('kurucz_cd23_chianti_H_He.h5')
marcs_model_fv, marcs_abundances_all, temps = read_marcs_to_fv('data/marcs/sun.mod', adata, 30)
adata.prepare_atom_data(marcs_abundances_all.index.tolist())

## Wavelengths and Frequencies

In [ ]:
tracing_lambda = np.arange(1000, 10000, 10) * u.Angstrom
tracing_nus = tracing_lambda.to(u.Hz, u.spectral())

## Plasma

In [ ]:
splasma = create_splasma(marcs_model_fv, marcs_abundances_all, adata, tracing_nus)

## Optical Depths

In [ ]:
tau_nus = calc_tau_nus(splasma, marcs_model_fv,tracing_nus)

In [ ]:
tau_h_minus = calc_tau_h_minus(
    splasma,
    marcs_model_fv,
    tracing_nus,
    wbr_fpath="data/wishart_broad_reinhardt_cross_section.dat",
)

In [ ]:
tau_e = calc_tau_e(splasma,marcs_model_fv,tracing_nus)

In [ ]:
tau_photo_H = calc_tau_photo(splasma,marcs_model_fv,tracing_nus,(1,0,1),7.91e-18,8.22e14)

In [ ]:
all_taus = [tau_nus, tau_h_minus, tau_e, tau_photo_H]

## Source Function

In [ ]:
bb = bb_nu(tracing_nus, temps)

In [ ]:
plt.plot(tracing_nus, bb[0])

In [ ]:
# bb shape: (56, 800000), calculated at each cell boundary
bb_prefactor = (2 * const.h.cgs * tracing_nus ** 3) / const.c.cgs ** 2
bb = bb_prefactor / (
    np.exp(
        (
            (const.h.cgs * tracing_nus)
            / (const.k_B.cgs * temps * u.K)
        ).value
    )
    - 1
)

In [ ]:
tw_cm = tracing_lambda.to("cm")

In [ ]:
bbw_prefactor = (2 * const.h.cgs * const.c.cgs ** 2) / (tracing_lambda) ** 5
bbw = bbw_prefactor / (
    np.exp(
        (
            (const.h.cgs * const.c.cgs)
            / (const.k_B.cgs * tracing_lambda * temps * u.K)
        )
    )
    - 1
) * 1e-7

In [ ]:
tw_cm

In [ ]:
plt.plot(tracing_lambda,bbw[0])

In [ ]:
plt.plot(tracing_lambda,bb[0]*tracing_nus/tracing_lambda**2)

In [ ]:
for i in range(55):
    plt.plot(tracing_lambda,bbw[i])

In [ ]:
for i in range(55):
    plt.plot(tracing_nus,bb[i])

In [ ]:
bb.shape

In [ ]:
source = bb[1:].value 
delta_source = bb.diff(axis=0).value  # for cells, not boundary

## Ray tracing

In [ ]:
# I_nu (specific intensity) is calculated at each cell boundary, shape: (56,800000)
I_nu = np.ones((delta_tau_lines.shape[1] + 1, len(tracing_nus))) * -99
I_nu[0] = bb[0]  # the innermost boundary is photosphere
num_of_shells = len(I_nu) - 1
tau_photo_H_fn = calc_tau_photo(splasma,marcs_model_fv,tracing_nus,(1,0,1),7.91e-18,8.22e14)

for i in range(len(tracing_nus)):  # iterating over nus (columns)
    
    nu = tracing_nus[i]

    for j in range(num_of_shells):  # iterating over cells/shells (rows)
        cur_tau_h_minus = tau_h_minus[j,i]
        curr_tau_e = tau_e[j,i]
        curr_tau_photo = tau_photo_H_fn[j,i]
        curr_tau_nus = tau_nus[j,i]
        
        delta_tau_nu = tau_nus[j,i] + cur_tau_h_minus + curr_tau_photo + curr_tau_e #+ tauross[j] seems to improve
        w0, w1 = calc_weights(delta_tau_nu)

        if tau_nus[j,i] == 0:
            second_term = 0
        else:
            second_term = w1 * delta_source[j, i] / delta_tau_nu

        I_nu[j + 1, i] = (1 - w0) * I_nu[j, i] + w0 * source[j, i] + second_term # van Noort 2001 eq 14

## Plotting

In [ ]:
I_lambda = (I_nu[55])*tracing_nus/tracing_lambda

cur_bb = bb[0]*tracing_nus/tracing_lambda #TODO: check why 0, shouldn't it be 55 like I_nu?

plt.plot(tracing_lambda, I_lambda, label="I")
plt.plot(tracing_lambda, cur_bb, label="BB")
# plt.xlim(4330,4350) # to see 1st feature
# plt.xlim(6500, 6600) # to see last feature
#plt.xlim(2000,10000) 
plt.legend()

In [ ]:
plt.figure(figsize=(10,6))
flux = 2*np.pi*1e-5*ndimage.gaussian_filter1d(I_lambda, 1)
plt.plot(tracing_lambda, flux, label="F")
plt.plot(sun_spec.wave, sun_spec.flux, label="$F_{sun}$")
plt.plot(tracing_lambda,bbw[14], label="BB of some temp")

plt.xlim((1000,10000))
plt.legend()

## Scratch Work

In [ ]:
kappaross_diff = -np.diff(marcs_model_fv.kappaross)
kappaross_diff = kappaross_diff
kappaross_diff = np.append(kappaross_diff,marcs_model_fv.kappaross[54])
kappaross_diff

In [ ]:
tauross = kappaross_diff*marcs_model_fv.density*marcs_model_fv.cell_length
tauross